In [ ]:
import requests
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from datetime import date
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()
username = os.getenv("ARCGIS_USER")
password = os.getenv("ARCGIS_PASS")

# 1. Connect to ArcGIS 
gis = GIS("https://www.arcgis.com", "zhiquanchng", "Q@JJMpTFdK8k-u_o")

# 2. Link the Butterfly Layer
layer_url = "https://services7.arcgis.com/NoGQYZi3a1tVAyzH/arcgis/rest/services/butterfly_data_for_arcGIS/FeatureServer/0"
butterfly_layer = FeatureLayer(layer_url)

# 3. iNaturalist Settings
today = date.today().isoformat()
inat_url = f"https://api.inaturalist.org/v1/observations?place_id=6734&taxon_id=47224&d1={today}&per_page=200"

In [ ]:
# --- THE SYNC LOGIC ---
print("Fetching from iNaturalist...")
data = requests.get(inat_url).json()

cleaned_features = []

for obs in data['results']:
    # Handle the 'Confidence' and 'Transect' as blank/None
    # iNaturalist doesn't have a direct 'transect' field in standard obs
    # Get the username safely
    user_info = obs.get('user', {})
    inat_username = user_info.get('login', 'Unknown')    

    # Safely get taxon name
    taxon_name = obs.get('taxon', {}).get('preferred_common_name', "Unknown")
    
    # Extract date details safely
    date_details = obs.get('observed_on_details', {})
    
    feature = {
        "attributes": {
            "ID": None, # Leave blank 
            "iNat_ID" : str(obs['id']),
            "Username" : inat_username,
            "Year": date_details.get('year'),
            "Month": date_details.get('month'),
            "Park": obs.get('place_guess', "Singapore"),
            "Latitude": obs['geojson']['coordinates'][1],
            "Longitude": obs['geojson']['coordinates'][0],
            "Transect": None,  # Leave blank
            "Start_Time": obs.get('time_observed_at'),
            "Confidence": None, # Leave blank
            "Species": taxon_name,
            "Count_": 1
        },
        "geometry": {
            "x": obs['geojson']['coordinates'][0],
            "y": obs['geojson']['coordinates'][1],
            "spatialReference": {"wkid": 4326}
        }
    }
    cleaned_features.append(feature)

# Push the cleaned list to ArcGIS
if cleaned_features:
    butterfly_layer.edit_features(adds=cleaned_features)
    print(f"Successfully added {len(cleaned_features)} sightings!")

Fetching from iNaturalist...
Successfully added 34 sightings!
